## Discount factor
**Discount factor**  - это параметр, используемый в теории управления и обучении с подкреплением для оценки стоимости будущих вознаграждений. Он обозначается символом $\gamma$ (гамма).

В контексте задачи обучения с подкреплением, такой как у **DQN** алгоритма, фактор дисконтирования применяется к будущим вознаграждениям, чтобы учесть уменьшение их значения с течением времени. Фактор дисконтирования позволяет агенту принимать во внимание не только мгновенные вознаграждения, но и будущие вознаграждения, приводя к более долгосрочной стратегии.

**Формула для дисконтирования будущих вознаграждений** выглядит следующим образом:

 $G_t$ = $R_{t+1}$ + $\gamma$ $R_{t+2}$ + $\gamma$^2 $R_{t+3}$ + $\ldots$ = $\sum_{k=0}^{\infty}$ $\gamma$^k $R_{t+k+1}$

где:
- $G_t$ - дисконтированная сумма вознаграждений (вознаграждение с учетом будущих шагов);
- $R_{t+k+1}$ - вознаграждение, полученное на шаге $t+k+1$;
- $\gamma$ - фактор дисконтирования,  $0 \leq \gamma \leq 1$.

Фактор дисконтирования имеет важное значение при принятии решений в условиях неопределенности, где агенту нужно учитывать как мгновенные, так и будущие вознаграждения, с учетом их временного удаления. Выбор подходящего значения для фактора дисконтирования может влиять на стратегии обучения.


## Long-term и Short-term стратегия

Фактор дисконтирования оказывает сильное влияние на долгосрочность стратегии.
В контексте обучения с подкреплением и стратегий агента, термины "краткосрочные" и "долгосрочные" относятся к временным характеристикам принятия решений агентом.

**Краткосрочные стратегии**:

> Агент, придерживающийся краткосрочной стратегии, ориентируется в основном на текущую информацию и мгновенные вознаграждения.
Такой агент может принимать решения, которые приносят максимальное мгновенное вознаграждение, не уделяя большого внимания будущим шагам. Такой агент может принимать решения, которые приносят максимальное мгновенное вознаграждение, не уделяя большого внимания будущим шагам.




**Долгосрочные стратегии**:


> Агент, придерживающийся долгосрочной стратегии, учитывает будущие вознаграждения и последствия своих действий на протяжении более длительного времени.
Такой агент может предпочесть действия, которые могут не приносить максимальное мгновенное вознаграждение, но могут способствовать достижению более высоких наград в будущем.

Когда говорят о краткосрочных или долгосрочных стратегиях в контексте обучения с подкреплением, обычно имеется в виду, как агент принимает решения, оптимизируя сумму будущих вознаграждений. Фактор дисконтирования γ в уравнении обучения с подкреплением регулирует степень учета будущих вознаграждений, и, таким образом, влияет на то, насколько агент ориентирован на краткосрочные или долгосрочные перспективы.

## Метрики для оценки модели
В данной работе мы рассмотрим некоторые метрики, которые могут быть использованы для оценки обучения модели.
Мы будем решать задачу в окружении Taxi-v3, а в качестве модели используем уже знакомый DQN. При этом мы обучим модель несколько разных с разным параметром Discount factor.

Для оценки модели мы рассмотрим следущие метрики:

*   Q-values, их поведение и сходимость.
*   Средняя награда за эпизод.
*   Время затрачиваемое на обучение модели.








### Установка зависимостей

На первом шаге мы начинаем с установки необходимых библиотек. Пакет [gymnasium](https://en.wikipedia.org/wiki/Q-learning#Deep_Q-learning) предоставляет различные среды для обучения с подкреплением, в то время как [stable-baselines3](https://en.wikipedia.org/wiki/Q-learning#Deep_Q-learning) предоставляет реализации различных алгоритмов обучения с подкреплением.

In [1]:
!sudo apt install xvfb
!pip install gymnasium
!pip install stable-baselines3
!pip install PyVirtualDisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 7,816 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

Импортируем пакет, которые будут использованы в данной работе


In [2]:
# Импорт библиотек
import torch as th
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
import time

Q-значения (Q-values) представляют собой оценки или оценочные функции, используемые в контексте обучения с подкреплением для измерения ожидаемой полезности (ценности) принятия определенного действия в конкретном состоянии.

Для получения значений для нашей модели реализуем функцию, которая их извлекает из q-сети.

In [3]:
ENV_NAME = "MountainCar-v0"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def q_values(model: DQN, obs: np.ndarray) -> np.ndarray:
  q_net = model.q_net
  obs_tensor = th.tensor(obs, dtype=th.float32)
  obs_tensor = obs_tensor.unsqueeze(0)
  q_values = model.q_net.forward(obs_tensor)
  return q_values.detach().numpy()[0]

Для наглядности результатов Q-values необходимо создать функцию для составления графика по значениям.

In [5]:
def plot_q_values(q_values_list):
  q1_values, q2_values, q3_values = zip(*q_values_list)

  plt.figure(figsize=(10, 6))
  plt.plot(q1_values, label='Q1 Values')
  plt.plot(q2_values, label='Q2 Values')
  plt.plot(q3_values, label='Q3 Values')
  plt.xlabel('Time')
  plt.ylabel('Q-Values')
  plt.title('Convergence of Q-Values over Time')
  plt.legend()
  plt.show()

Следующий блок кода служит для следующих целей:

*   Создание окружения
*   Создание модели
*   Обучение модели
*   Оценка модели до обучения
*   Оценка модели после обучения
*   Измерение времени обучени


In [6]:
def mean_reward(discount_factor):
  env = gym.make(ENV_NAME, render_mode="human")
  model = DQN("MlpPolicy", env, verbose=1, gamma=discount_factor)
  n_eval_episodes = 250

  mean_reward, std_reward = evaluate_policy(model, gym.make(ENV_NAME, render_mode="human"), deterministic=True, n_eval_episodes=n_eval_episodes)
  print(f"До обучения модели с discount_factor = {discount_factor}, mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

  start_time = time.time()

  model.learn(total_timesteps=10000, log_interval=100)

  end_time = time.time()

  mean_reward, std_reward = evaluate_policy(model, gym.make(ENV_NAME, render_mode="human"), deterministic=True, n_eval_episodes=n_eval_episodes)
  print(f"После обучения модели с discount_factor = {discount_factor}, mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
  model.save(f"dqn_cartpole_{discount_factor}")
  del model

  learn_time = end_time - start_time
  return env, learn_time

Q-values необходимо вычислять для некоторого набора предсказаний модели, реализуем для этого функцию

In [7]:
def q_values_calculation(discount_factor, env):
  model = DQN.load(f"dqn_cartpole_{discount_factor}")

  action_str = ["left","no", "right"]
  q_values_list = []

  obs, info = env.reset()
  for _ in range(100):
      q_val = q_values(model,obs)
      q_values_list.append(q_val)
      action, _states = model.predict(obs, deterministic=True)

      print(f"Q-value состояния left={q_val[0]:.2f} no={q_val[1]:.2f} right={q_val[2]:.2f}")
      print(f"Действие: {action_str[action]}")

      obs, reward, terminated, truncated, info = env.step(action)

  return q_values_list

Изменение значения discount factor (γ) в DQN может оказать существенное воздействие на обучение агента и его стратегию.

Воздействие на краткосрочные или долгосрочные стратегии:

*   При более высоком значении γ (ближе к 1) агент будет больше учитывать будущие вознаграждения, что может способствовать формированию более долгосрочных стратегий.

*  При более низком значении γ (ближе к 0) агент будет более склонен к учету только мгновенных вознаграждений, что может привести к формированию краткосрочных стратегий.


Влияние на стабильность обучения:

*   Высокий γ может сделать обучение более стабильным, особенно в средах с большим количеством шума или случайности.
*   Низкий γ может привести к более нестабильному обучению, так как агент будет менее чувствителен к долгосрочным последствиям своих действий.


Влияние на временные рамки обучения:


*   Различные значения γ могут потребовать различных временных рамок обучения для достижения оптимальных стратегий.
*   Изменение γ может потребовать перенастройки гиперпараметров и количества шагов обучения.


Адаптивность к задаче:
*  Эффективное значение γ может зависеть от конкретной задачи. Например, в задачах с большой неопределенностью и случайностью, более высокий γ может быть полезен.


Экспериментальные результаты:
*  Чтобы определить оптимальное значение γ для вашей конкретной задачи, часто требуется провести эксперименты с различными значениями, а затем анализировать результаты.

Рассмотрим 3 значения discount factor

In [8]:
discount_factors = [0.01,0.5,0.99]

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[0])
q_vals = q_values_calculation(discount_factors[0], environment)
print(f'Время обучения модели при discount_factor = {discount_factors[0]} : {time_to_lrn} секунд.')

In [ ]:
plot_q_values(q_vals)

#### Закрытие среды после тестирования
По завершении тестирования мы закрываем среду с помощью env.close().

In [ ]:
# Закрытие среды
environment.close()

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[1])
q_vals = q_values_calculation(discount_factors[1], environment)
print(f'Время обучения модели при discount_factor = {discount_factors[1]} : {time_to_lrn} секунд.')

In [ ]:
plot_q_values(q_vals)

In [ ]:
environment.close()

In [ ]:
environment, time_to_lrn = mean_reward(discount_factors[2])
q_vals = q_values_calculation(discount_factors[2], environment)
print(f'Время обучения модели при discount_factor = {discount_factors[2]} : {time_to_lrn} секунд.')

In [ ]:
plot_q_values(q_vals)

In [ ]:
environment.close()